The purpose of this project is to apply the QR algorithm to estimate the eigenvectors of a matrix. Since I am going to need a square matrix, I decided to construct a stock return variance-covairance matrix in Microsoft Excel using stock return data that I downloaded from Bloomberg. I am using monthly return data for Amazon, Microsoft, Apple, Facebook, and Johnson and Johnson over the four-year period from January 2016 throughout December 2019.  

The covariance matrix has several properties that play an important role in determining its eigenvalues and eigenvectors. Since it is symmetric, it will have all real eigenvalues. Moreover, since it is both symmetric and has real entries, the eigenvectors will be orthogonal.  

Any square, $n\times n$ matrix will have $n$ eigenvalues and $n$ eigenvectors. Let $\mathbf{A}$ be an arbitrary $n\times n$ matrix. Then, if $\lambda$ is a scalar and $\mathbf{x}$ is a nonzero vector such that
$$
\mathbf{Ax}=\lambda \mathbf{x},
$$
we say that $\lambda$ is an eigenvalue of $\mathbf{A}$ and $\mathbf{x}$ is a corresponding eigenvector. Equivalently, we have $\mathbf{Ax}-\lambda \mathbf{x}=(\mathbf{A}-\lambda \mathbf{I})\mathbf{x}=0$, and therefore $\lambda$ is an eigenvalue of $\mathbf{A}$ if det$(\mathbf{A}-\lambda \mathbf{I})=0$. The expression det$(\mathbf{A}-\lambda \mathbf{I})$ is called the characteristic polynomial. This $n$-degree polynomial will have $n$ roots, hence $\mathbf{A}$ can have at most $n$ eigenvalues. Consequently, computing the eigenvalues of matrices of orders greater than four cannot be done in a finite number of steps, since there is no formula to find roots of a polynomial of degree greater than four. Therefore, I will need to use an iterative algorithm to compute the eigenvalues for this $5\times 5$ stock return covariance matrix.  

The QR iteration algorithm produces a sequence of matrices
$$
\mathbf{A}_k=\mathbf{Q}_k^T \mathbf{A}\mathbf{Q}_k
$$
that will eventually converge to an upper triangular matrix after $k=1,2,…,k$ iterations, where $\mathbf{Q}_k$ is square and orthogonal (i.e., $\mathbf{Q}_k^T=\mathbf{Q}_k =\mathbf{I}$). Since each $\mathbf{A}_k$ is similar to $\mathbf{A}$, it will have the same eigenvalues. Thus this system will actually converge to an upper triangular matrix with the eigenvalues of $\mathbf{A}$ in its diagonal entries. We begin by setting 
$$
\mathbf{A}=\mathbf{Q}_0 \mathbf{R}_0,
$$
where $\mathbf{Q}_0$ is orthogonal and $\mathbf{R}_0$ is upper-triangular, and every subsequent iteration produces a new QR factorization.  

Next, we compute 
$$
\mathbf{A}_1=\mathbf{R}_0 \mathbf{Q}_0 =\mathbf{Q}_0^T \mathbf{Q}_0 \mathbf{R}_0 \mathbf{Q}_0 = \mathbf{Q}_0^T \mathbf{A} \mathbf{Q}_0 =\mathbf{Q}_0^{-1} \mathbf{A} \mathbf{Q}_0,
$$
which is obviously similar to $\mathbf{A}$. For each step, we find a QR factorization for $\mathbf{A}_k= \mathbf{Q}_k \mathbf{R}_k$, then set $\mathbf{A}_{k+1}= \mathbf{R}_k \mathbf{Q}_k$, and repeat until we get a matrix $\mathbf{A}_k$ that is upper-triangular. When $\mathbf{A}_k$ converges, the eigenvalues of $\mathbf{A}$ are the diagonal entries, and the eigenvectors can be found by taking the product of the orthogonal matrices $\mathbf{Q}_k$ generated by each step of the algorithm.  

I am going to write a `loop` to take run through this algorighm, and then check whether or not they agree with `Numpy`'s `eig` function.

In [1]:
import numpy as np
import numpy.linalg as LA

In [6]:
A = np.array([[59.57674104, 21.98446019, 23.48992858, 29.75478007,  9.16373926],
               [21.98446019, 23.91693356, 22.32054364, 16.44214813,  6.66092005],
               [23.48992858, 22.32054364, 60.46819669, 24.76732211,  7.89820125],
               [29.75478007, 16.44214813, 24.76732211, 54.00538468,  3.93900408],
               [ 9.16373926,  6.66092005,  7.89820125,  3.93900408, 17.14454932]])

Here is the eigenvalue decomposition:

In [7]:
LA.eig(A)

(array([124.36422893,  37.68186416,  28.33666536,  10.07157865,
         14.65746819]),
 array([[-0.56309748, -0.54292747, -0.51010673,  0.21539744, -0.2855627 ],
        [-0.33550751,  0.07764034, -0.19172374, -0.90202671,  0.17605829],
        [-0.53898299,  0.78493118, -0.03466974,  0.23881301, -0.18747461],
        [-0.51340851, -0.28669906,  0.79024585,  0.03140843,  0.16952985],
        [-0.12753425,  0.02980653, -0.27810546,  0.28624049,  0.90750775]]))

Now, I will begin the QR Iteration algorithm by setting up the initial values ($\mathbf{A}_0=\mathbf{Q}_0 \mathbf{R}_0$):

In [17]:
A0 = A
Q0,R0 = LA.qr(A0)
q0 = Q0

In [18]:
A1 = R0.dot(Q0)
A1

array([[113.77602487,  23.09878726,  12.43938964,  12.59446812,
         -1.78619454],
       [ 23.09878726,  34.0159582 ,  16.48239948,   1.13406569,
         -1.45746863],
       [ 12.43938964,  16.48239948,  22.87608802,   1.37453023,
          1.30873722],
       [ 12.59446812,   1.13406569,   1.37453023,  30.26247171,
          1.69529622],
       [ -1.78619454,  -1.45746863,   1.30873722,   1.69529622,
         14.18126248]])

In [19]:
Q1,R1 = LA.qr(A1)
q1 = Q1

In [20]:
A2 = R1.dot(Q1)
A2

array([[123.47544506,   8.14604263,   1.68578927,   3.01970922,
          0.20561056],
       [  8.14604263,  36.99587169,   5.34895942,  -2.03637633,
          0.42028538],
       [  1.68578927,   5.34895942,  13.06235263,   2.31291929,
         -2.21785054],
       [  3.01970922,  -2.03637633,   2.31291929,  28.26519373,
         -0.64809965],
       [  0.20561056,   0.42028538,  -2.21785054,  -0.64809965,
         13.31294218]])

In [21]:
Q2,R2 = LA.qr(A2)
q2 = Q2

In [22]:
A3 = R2.dot(Q2)
A3

array([[ 1.24290828e+02,  2.42779807e+00,  3.72684834e-01,
         6.54584968e-01, -2.10311607e-02],
       [ 2.42779807e+00,  3.74964966e+01,  1.20729184e+00,
        -1.66749164e+00, -1.38476042e-01],
       [ 3.72684834e-01,  1.20729184e+00,  1.47269783e+01,
         5.62664825e+00,  2.29572713e+00],
       [ 6.54584968e-01, -1.66749164e+00,  5.62664825e+00,
         2.61536508e+01, -4.40128168e-01],
       [-2.10311607e-02, -1.38476042e-01,  2.29572713e+00,
        -4.40128168e-01,  1.24438517e+01]])

I am now going to speed things up a bit by using `loop`s. Notice that I am creating the object `q` here, which is equal to the product of all orthogonal matrices generated by each iteration. That is,
$$
\mathbf{q} = \mathbf{Q}_0 \cdot \mathbf{Q}_1 \cdots \mathbf{Q}_k
$$
for $k$ iterations. When $k$ converges to an upper-triangular matrix, $\mathbf{q}$ will give us its eigenvectors.

In [26]:
Ak = A3
q = q0.dot(q1).dot(q2)

for k in range(10):
    Qk,Rk = LA.qr(Ak)
    qk = Qk
    q = q.dot(qk)
    Ak = Rk.dot(Qk)

print(np.array_str(Ak))

[[ 1.24364229e+02  1.51647359e-05  2.83801443e-07  6.18156040e-10
  -3.65145488e-13]
 [ 1.51647359e-05  3.76814219e+01 -6.42875322e-02 -2.35968294e-04
  -3.71461712e-07]
 [ 2.83801427e-07 -6.42875322e-02  2.83370397e+01  3.04765334e-02
   2.71996838e-04]
 [ 6.18146438e-10 -2.35968294e-04  3.04765334e-02  1.46548667e+01
  -1.10608968e-01]
 [-3.66318211e-13 -3.71461707e-07  2.71996838e-04 -1.10608968e-01
   1.00742480e+01]]


In [27]:
for k in range(100): 
    Qk,Rk = LA.qr(Ak)
    qk = Qk
    q = q.dot(qk)
    Ak = Rk.dot(Qk)

print(np.array_str(Ak))

[[ 1.24364229e+002  2.02784602e-014  1.65383951e-014  9.53455893e-015
   1.40314252e-015]
 [ 2.11085530e-057  3.76818642e+001 -2.57460500e-014 -5.18712035e-015
  -5.26559430e-015]
 [ 1.65330076e-071 -2.69020861e-014  2.83366654e+001  1.18324711e-015
   2.51535288e-015]
 [ 8.45902112e-103 -2.31942402e-045  7.15708574e-031  1.46574682e+001
   6.03755173e-016]
 [-2.53624756e-122 -1.84742856e-064  3.23264285e-049 -5.59633015e-018
   1.00715786e+001]]


In [28]:
for k in range(1000): 
    Qk,Rk = LA.qr(Ak)
    qk = Qk
    q = q.dot(qk)
    Ak = Rk.dot(Qk)

print(np.array_str(Ak, suppress_small=True))

[[124.36422893   0.           0.           0.           0.        ]
 [  0.          37.68186416   0.          -0.          -0.        ]
 [  0.          -0.          28.33666536   0.           0.        ]
 [  0.           0.           0.          14.65746819   0.        ]
 [  0.           0.           0.          -0.          10.07157865]]


In [29]:
for k in range(1000): 
    Qk,Rk = LA.qr(Ak)
    qk = Qk
    q = q.dot(qk)
    Ak = Rk.dot(Qk)

print(np.array_str(Ak, suppress_small=True))

[[124.36422893   0.           0.           0.           0.        ]
 [  0.          37.68186416   0.          -0.          -0.        ]
 [  0.          -0.          28.33666536   0.           0.        ]
 [  0.           0.           0.          14.65746819   0.        ]
 [  0.           0.           0.          -0.          10.07157865]]


After a little over $2,000$ iterations it appears that $k$ has converged. The diagonal entries appear to agree with the eigenvalues in the decomposition above, so everything looks good so far. Here are the eigenvectors I obtained from the QR iterations:

In [30]:
q

array([[-0.56309748,  0.54292747,  0.51010673,  0.2855627 ,  0.21539744],
       [-0.33550751, -0.07764034,  0.19172374, -0.17605829, -0.90202671],
       [-0.53898299, -0.78493118,  0.03466974,  0.18747461,  0.23881301],
       [-0.51340851,  0.28669906, -0.79024585, -0.16952985,  0.03140843],
       [-0.12753425, -0.02980653,  0.27810546, -0.90750775,  0.28624049]])

These eigenvectors also agree with the output of the `eig()` function above, so my work is correct.